# Persian Stance Classification - Deep Learning

In [1]:
from tqdm import tqdm
import numpy as np
import pandas as pd
import numpy as np
import numpy as np
import os.path as path

# Mount Google Drive

In [2]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# input files
cleaned_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/Clean_Claim_Body.csv"
train_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/train_data.csv"
test_path = "/content/drive/MyDrive/Stance Detection Project/dataset cleaned/test_data.csv"

FEATURES_DIR = "/content/drive/MyDrive/Stance Detection Project/Baseline Zarharan/features/"

sequences_file = FEATURES_DIR + "sequences.npy"

# output files
deep_model_path = "/content/drive/MyDrive/persian_stance_deep_data/Baseline Zarharan/models/model_v1.h5"

# Read Cleaned Data from CSV File

In [4]:
import pandas as pd
dataset_clean = pd.read_csv(cleaned_path, index_col = 0, )

In [5]:
clean_claim = dataset_clean['claim']
clean_body = dataset_clean['body']

In [6]:
dataset_clean.head()

,claim,body,label
0,کلاهبرداری از رانندگان با شگرد نشت بنزین !,به گزارش خبرنگار گروه جامعه خبرگزاری میزان،29 ...,Discuss
1,تجاوز به دختر بازداشت شده و واژگونی ون گشت ارش...,انتشار کلیپ واژگونی ماشین گشت ارشاد توسط مردم ...,Discuss
2,تعظیم 20 دقیقه ای وزیر نیرو ژاپن به علت قطع بر...,وزیر نیروی ژاپن به علت قطع شدن برق؛ به همان مد...,Agree
3,سرمربیگری گاس هیدینک برای تراکتورسازی,به تازگی محمد تقوی استعفای خود را از سرمربیگری...,Discuss
4,کشف موجود عجیبی شبیه انسان در یک حفاری در پاکس...,پس از 20 سال حفاری با دقتی باورنکردنی، سرانجام...,Unrelated


# Read Data Train and Test from CSV Files

In [7]:
data_train = pd.read_csv(train_path, index_col = 0, )
data_test = pd.read_csv(test_path, index_col = 0, )

In [8]:
len(data_train), len(data_test)

(1597, 400)

# Load Features

In [9]:
X_LSTM = np.load(sequences_file)
y_LSTM = dataset_clean['label'].tolist()

In [10]:
X_LSTM.shape

(1997, 100)

In [11]:
claims = dataset_clean['claim'].tolist()
bodies = dataset_clean['body'].tolist()

concatenated = []
for i in range(len(claims)):
    concatenated.append(claims[i] + ". " + bodies[i])

X_BERT = concatenated
y_BERT = dataset_clean['label'].tolist()

# Deep Learning Model

In [12]:
import numpy as np
import os.path as path
import pickle

from keras.layers.core import Dense
from keras.layers import LSTM
from keras import optimizers
from keras.models import Model, load_model
from keras.layers import concatenate
from keras.layers import Embedding, Input

In [13]:
from sklearn.model_selection import train_test_split

X_train_LSTM, X_test_LSTM, y_train, y_test = train_test_split(
    X_LSTM, y_LSTM, test_size=0.2, random_state=42)

In [14]:
def convert_data_to_one_hot(y_train):

    y_train_temp = np.zeros((y_train.size, y_train.max() + 1), dtype=np.int)
    y_train_temp[np.arange(y_train.size), y_train] = 1

    return y_train_temp

In [15]:
from sklearn import preprocessing
import keras

le = preprocessing.LabelEncoder()
le.fit(y_train)
print(list(le.classes_))
y_train = le.transform(y_train)

le = preprocessing.LabelEncoder()
le.fit(y_test)
print(list(le.classes_))
y_test = le.transform(y_test)

print(y_test[0:10])

['Agree', 'Disagree', 'Discuss', 'Unrelated']
['Agree', 'Disagree', 'Discuss', 'Unrelated']
[2 2 1 3 2 0 1 2 3 2]


In [16]:
def calculate_class_weight(y_train, no_classes=2):
    # https://datascience.stackexchange.com/questions/13490/how-to-set-class-weights-for-imbalanced-classes-in-keras
    from sklearn.utils import class_weight

    class_weight_list = class_weight.compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
    class_weights = {}
    for i in range(no_classes):
        class_weights[i] = class_weight_list[i]
    print(class_weights)
    return class_weights

In [17]:
y_train_one_hot = convert_data_to_one_hot(y_train)
y_test_one_hot = convert_data_to_one_hot(y_test)

param_dict="single_flat_LSTM_50d_100"

PARAM_DICT_FILENAME = param_dict + "_param_dict.pkl"


# load feature dict for LSTM_1000_GloVe
with open(FEATURES_DIR+PARAM_DICT_FILENAME, "rb") as f:
  param_dict = pickle.load(f)

# load parameters needed for embedding layer
EMBEDDING_DIM = param_dict["EMBEDDING_DIM"] # e.g. 50
MAX_SEQ_LENGTH = param_dict["MAX_SEQ_LENGTH"] # e.g. 100

# load embeddings
EMBEDDING_FILE = np.load(FEATURES_DIR+param_dict["EMBEDDING_FILE"])

print("EMBEDDING_FILE.shape = " + str(EMBEDDING_FILE.shape))

# calc cass weights
class_weights = calculate_class_weight(y_train, no_classes=4)

EMBEDDING_FILE.shape = (48758, 50)
{0: 3.6295454545454544, 1: 2.4344512195121952, 2: 0.45733104238258876, 3: 0.8872222222222222}


<ipython-input-14-d1e925b257f1>:3: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  y_train_temp = np.zeros((y_train.size, y_train.max() + 1), dtype=np.int)


In [18]:
kernel_initializer = 'glorot_uniform'
regularizer = None
batch_size=200
dense_activity_regularizer=None
LSTM_implementation = 2


################
# CLAIMS LAYER #
################
lstm_input = Input(shape=(MAX_SEQ_LENGTH,), dtype='int32', name='lstm_input') # receive sequences of MAX_SEQ_LENGTH_CLAIMS integers
embedding = Embedding(input_dim=len(EMBEDDING_FILE), # lookup table size
                                    output_dim=EMBEDDING_DIM, # output dim for each number in a sequence
                                    weights=[EMBEDDING_FILE],
                                    input_length=MAX_SEQ_LENGTH, # receive sequences of MAX_SEQ_LENGTH_CLAIMS integers
                                    mask_zero=True,
                                    trainable=True)(lstm_input)
data_LSTM = LSTM(
            100, return_sequences=True, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_SEQ_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_SEQ_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
            )(embedding)
data_LSTM = LSTM(
            100, return_sequences=False, stateful=False, dropout=0.2,
            batch_input_shape=(batch_size, MAX_SEQ_LENGTH, EMBEDDING_DIM),
            input_shape=(MAX_SEQ_LENGTH, EMBEDDING_DIM), implementation=LSTM_implementation
            )(data_LSTM)


###############
# MERGE LAYER #
###############
merged = data_LSTM

dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(merged)
dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(dense_mid)
dense_mid = Dense(600, kernel_regularizer=regularizer, kernel_initializer=kernel_initializer,
                          activity_regularizer=dense_activity_regularizer, activation='relu')(dense_mid)
dense_out = Dense(4,activation='softmax', name='dense_out')(dense_mid)

# build model
model = Model(inputs=[lstm_input], outputs=[dense_out])

# print summary
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_input (InputLayer)     [(None, 100)]             0         
                                                                 
 embedding (Embedding)       (None, 100, 50)           2437900   
                                                                 
 lstm (LSTM)                 (None, 100, 100)          60400     
                                                                 
 lstm_1 (LSTM)               (None, 100)               80400     
                                                                 
 dense (Dense)               (None, 600)               60600     
                                                                 
 dense_1 (Dense)             (None, 600)               360600    
                                                                 
 dense_2 (Dense)             (None, 600)               360600

In [19]:
LABELS = ['Agree', 'Disagree', 'Discuss', 'Unrelated']
LABELS_RELATED = ['unrelated','related']
RELATED = LABELS[0:3]

def score_submission(gold_labels, test_labels):
    score = 0.0
    cm = [[0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0],
          [0, 0, 0, 0]]

    for i, (g, t) in enumerate(zip(gold_labels, test_labels)):
        g_stance, t_stance = g, t
        if g_stance == t_stance:
            score += 0.25
            if g_stance != 'unrelated':
                score += 0.50
        if g_stance in RELATED and t_stance in RELATED:
            score += 0.25

        cm[LABELS.index(g_stance)][LABELS.index(t_stance)] += 1

    return score, cm

In [20]:
from keras.callbacks import Callback
import numpy as np
class EarlyStoppingOnF1(Callback):
    """
    Prints some metrics after each epoch in order to observe overfitting
                https://github.com/fchollet/keras/issues/5794
                custom metrics: https://github.com/fchollet/keras/issues/2607
    """

    def __init__(self, epochs,
                 X_test_claims,
                 y_test, loss_filename, epsilon=0.0, min_epoch = 15, X_test_nt=None):
        self.epochs = epochs
        self.patience = 2
        self.counter = 0
        self.prev_score = 0
        self.epsilon = epsilon
        self.loss_filename = loss_filename
        self.min_epoch = min_epoch
        self.X_test_nt = X_test_nt
        #self.print_train_f1 = print_train_f1

        #self.X_train_claims = X_train_claims
        #self.X_train_orig_docs = X_train_orig_docs
        #self.X_train_evid = X_train_evid
        #self.y_train = y_train

        self.X_test_claims = X_test_claims
        self.y_test = y_test
        Callback.__init__(self)

    def on_epoch_end(self, epoch, logs={}):
        if epoch + 1 < self.epochs:
            from sklearn.metrics import f1_score

            # get prediction and convert into list
            if type(self.X_test_nt).__module__ == np.__name__:
                predicted_one_hot = self.model.predict([
                    self.X_test_claims,
                    self.X_test_nt
                ])
            else:
                predicted_one_hot = self.model.predict(self.X_test_claims)
            predict = np.argmax(predicted_one_hot, axis=-1)

            """
            predicted_one_hot_train = self.model.predict([self.X_train_claims, self.X_train_orig_docs, self.X_train_evid])
            predict_train = np.argmax(predicted_one_hot_train, axis=-1)


            # f1 for train data
            f1_macro_train = ""
            if self.print_train_f1 == True:
                f1_0_train = f1_score(self.y_train, predict_train, labels=[0], average=None)
                f1_1_train = f1_score(self.y_train, predict_train, labels=[1], average=None)
                f1_macro_train = (f1_0_train[0] + f1_1_train[0]) / 2
                print(" - train_f1_(macro): " + str(f1_macro_train))"""

            predicted = [LABELS[int(a)] for a in predict]
            actual = [LABELS[int(a)] for a in self.y_test]
            # calc FNC score
            fold_score, _ = score_submission(actual, predicted)
            max_fold_score, _ = score_submission(actual, actual)
            fnc_score = fold_score / max_fold_score
            print(" - fnc_score: " + str(fnc_score))

            # f1 for test data
            f1_0 = f1_score(self.y_test, predict, labels=[0], average=None)
            f1_1 = f1_score(self.y_test, predict, labels=[1], average=None)
            f1_2 = f1_score(self.y_test, predict, labels=[2], average=None)
            f1_3 = f1_score(self.y_test, predict, labels=[3], average=None)
            f1_macro = (f1_0[0] + f1_1[0] + f1_2[0] + f1_3[0]) / 4
            print(" - val_f1_(macro): " + str(f1_macro))
            print("\n")

            header = ""
            values = ""
            for key, value in logs.items():
                header = header + key + ";"
                values = values + str(value) + ";"
            if epoch == 0:
                values = "\n" + header + "val_f1_macro;" + "fnc_score;" + "\n" + values + str(f1_macro) + str(fnc_score) + ";"
            else:
                values += str(f1_macro) + ";" + str(fnc_score) + ";"
            append_to_loss_monitor_file(values, self.loss_filename)

            if epoch >= self.min_epoch-1:  # 9
                if f1_macro + self.epsilon <= self.prev_score:
                    self.counter += 1
                else:
                    self.counter = 0
                if self.counter >= 2:
                    self.model.stop_training = True
            #print("Counter at " + str(self.counter))
            self.prev_score = f1_macro
            #print("\n")

In [21]:
def append_to_loss_monitor_file(text, filepath):
    with open(filepath, 'a+') as the_file:
        the_file.write(text+"\n")

In [22]:
%%time

from tensorflow.keras import optimizers

lr = 0.001
optimizer_name = "RMS"
use_class_weights = True
epochs = 70
min_epoch = 20
save_folder = None
loss_filename = 'loss.h5'

# optimizers
if optimizer_name == "adagrad":
  optimizer = optimizers.Adagrad(lr=lr)
  print("Used optimizer: adagrad, lr="+str(lr))
elif optimizer_name == "adamax":
  optimizer = optimizers.Adamax(lr=lr)
  print("Used optimizer: adamax, lr="+str(lr))
elif optimizer_name == "nadam":
  optimizer = optimizers.Nadam(lr=lr)  # recommended to leave at default params
  print("Used optimizer: nadam, lr="+str(lr))
elif optimizer_name == "rms":
  optimizer = optimizers.RMSprop(lr=lr)  # recommended for RNNs
  print("Used optimizer: rms, lr="+str(lr))
elif optimizer_name == "SGD":
  optimizer = optimizers.SGD(lr=lr)  # recommended for RNNs
  print("Used optimizer: SGD, lr="+str(lr))
elif optimizer_name == "adadelta":
  optimizer = optimizers.Adadelta(lr)  # recommended for RNNs
  print("Used optimizer: adadelta, lr="+str(lr))
else:
  optimizer = optimizers.Adam(lr=lr)
  print("Used optimizer: Adam, lr=" + str(lr))

# compile model
model.compile(optimizer, 'kullback_leibler_divergence', # categorial_crossentropy
                           metrics=['accuracy'])
if use_class_weights == True:
  hist = model.fit(X_train_LSTM,
                             y_train_one_hot,
                             validation_data=(X_test_LSTM, y_test_one_hot),
                             batch_size=batch_size, epochs=epochs, verbose=1, class_weight=class_weights,
                              callbacks=[
                               EarlyStoppingOnF1(epochs,
                                                 X_test_LSTM, y_test,
                                                 loss_filename, epsilon=0.0, min_epoch=min_epoch),])
else:
  hist = model.fit(X_train_LSTM,
                             y_train_one_hot,
                             validation_data=(X_test_LSTM, y_test_one_hot),
                             batch_size=batch_size, epochs=epochs, verbose=1,
                           callbacks=[
                               EarlyStoppingOnF1(epochs,
                                                 X_test_LSTM, y_test,
                                                 loss_filename, epsilon=0.0, min_epoch=min_epoch),])



Used optimizer: Adam, lr=0.001
Epoch 1/70
13/13 [==============================] - 2s 8ms/step
 - fnc_score: 0.27459016393442626
 - val_f1_(macro): 0.1254681647940075


8/8 [==============================] - 29s 925ms/step - loss: 1.3915 - accuracy: 0.2893 - val_loss: 1.3719 - val_accuracy: 0.3350
Epoch 2/70
13/13 [==============================] - 0s 8ms/step
 - fnc_score: 0.2616120218579235
 - val_f1_(macro): 0.1166795267489712


8/8 [==============================] - 2s 271ms/step - loss: 1.3881 - accuracy: 0.1897 - val_loss: 1.3992 - val_accuracy: 0.1550
Epoch 3/70
13/13 [==============================] - 0s 11ms/step
 - fnc_score: 0.337431693989071
 - val_f1_(macro): 0.21393431060359042


8/8 [==============================] - 3s 369ms/step - loss: 1.3868 - accuracy: 0.1202 - val_loss: 1.3653 - val_accuracy: 0.3500
Epoch 4/70
13/13 [==============================] - 0s 8ms/step
 - fnc_score: 0.29030054644808745
 - val_f1_(macro): 0.18281818181818185


8/8 [========================

In [23]:
model.save("save.h5")

In [24]:
model.save(deep_model_path)

In [25]:
print("Loading model from:" + "save.h5")
model = load_model("save.h5")
predicted_one_hot = model.predict(X_test_LSTM)
predicted_int = np.argmax(predicted_one_hot, axis=-1)

Loading model from:save.h5
13/13 [==============================] - 3s 8ms/step


In [26]:
y_test_str = le.inverse_transform(y_test)
y_pred_str = le.inverse_transform(predicted_int)

In [27]:
# y_test_str

In [28]:
# y_pred_str

In [29]:
score, cm = score_submission(y_test_str, y_pred_str)
fold_score, _ = score_submission(y_test_str, y_pred_str)
max_fold_score, _ = score_submission(y_test_str, y_test_str)
score = fold_score / max_fold_score

print("FNC-1 score from restored model: " +  str(score) +"\n")

FNC-1 score from restored model: 0.4371584699453552



In [30]:
fold_score

160.0

In [31]:
max_fold_score

366.0

In [32]:
def print_confusion_matrix(cm):
    lines = []
    header = "|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format('', *LABELS)
    line_len = len(header)
    lines.append("-"*line_len)
    lines.append(header)
    lines.append("-"*line_len)

    hit = 0
    total = 0
    for i, row in enumerate(cm):
        hit += row[i]
        total += sum(row)
        lines.append("|{:^11}|{:^11}|{:^11}|{:^11}|{:^11}|".format(LABELS[i],
                                                                   *row))
        lines.append("-"*line_len)
    print('\n'.join(lines))


In [33]:
print_confusion_matrix(cm)

-------------------------------------------------------------
|           |   Agree   | Disagree  |  Discuss  | Unrelated |
-------------------------------------------------------------
|   Agree   |     3     |     2     |    10     |    12     |
-------------------------------------------------------------
| Disagree  |     2     |     6     |    22     |    12     |
-------------------------------------------------------------
|  Discuss  |     8     |     5     |    84     |    98     |
-------------------------------------------------------------
| Unrelated |    11     |     4     |    48     |    73     |
-------------------------------------------------------------


In [34]:
from sklearn.metrics import classification_report

print( classification_report(y_test_str, y_pred_str) )

# macro avg = unweighted mean of F1 values ***
# weighted avg = weighted mean of F1 values based on classes support/number of instances
# micro avg = a global F1 value based on sum of TPs, FPs, FNs

              precision    recall  f1-score   support

       Agree       0.12      0.11      0.12        27
    Disagree       0.35      0.14      0.20        42
     Discuss       0.51      0.43      0.47       195
   Unrelated       0.37      0.54      0.44       136

    accuracy                           0.41       400
   macro avg       0.34      0.31      0.31       400
weighted avg       0.42      0.41      0.41       400



# Results

In [ ]:
# Execute #1:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 2s 8ms/step
#  - fnc_score: 0.27459016393442626
#  - val_f1_(macro): 0.1254681647940075


# 8/8 [==============================] - 29s 925ms/step - loss: 1.3915 - accuracy: 0.2893 - val_loss: 1.3719 - val_accuracy: 0.3350
# Epoch 2/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2616120218579235
#  - val_f1_(macro): 0.1166795267489712


# 8/8 [==============================] - 2s 271ms/step - loss: 1.3881 - accuracy: 0.1897 - val_loss: 1.3992 - val_accuracy: 0.1550
# Epoch 3/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.337431693989071
#  - val_f1_(macro): 0.21393431060359042


# 8/8 [==============================] - 3s 369ms/step - loss: 1.3868 - accuracy: 0.1202 - val_loss: 1.3653 - val_accuracy: 0.3500
# Epoch 4/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.29030054644808745
#  - val_f1_(macro): 0.18281818181818185


# 8/8 [==============================] - 3s 354ms/step - loss: 1.3844 - accuracy: 0.2217 - val_loss: 1.3784 - val_accuracy: 0.3325
# Epoch 5/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2718579234972678
#  - val_f1_(macro): 0.12794006428436377


# 8/8 [==============================] - 2s 274ms/step - loss: 1.3872 - accuracy: 0.2248 - val_loss: 1.3395 - val_accuracy: 0.3225
# Epoch 6/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.23565573770491804
#  - val_f1_(macro): 0.03161592505854802


# 8/8 [==============================] - 2s 244ms/step - loss: 1.3723 - accuracy: 0.1559 - val_loss: 1.3853 - val_accuracy: 0.0675
# Epoch 7/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.5799180327868853
#  - val_f1_(macro): 0.1638655462184874


# 8/8 [==============================] - 2s 272ms/step - loss: 1.3865 - accuracy: 0.1985 - val_loss: 1.3659 - val_accuracy: 0.4875
# Epoch 8/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.273224043715847
#  - val_f1_(macro): 0.0999724844307362


# 8/8 [==============================] - 2s 245ms/step - loss: 1.3814 - accuracy: 0.5385 - val_loss: 1.3775 - val_accuracy: 0.1250
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.4842896174863388
#  - val_f1_(macro): 0.2745188276376612


# 8/8 [==============================] - 3s 336ms/step - loss: 1.3675 - accuracy: 0.2242 - val_loss: 1.3369 - val_accuracy: 0.4000
# Epoch 10/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2848360655737705
#  - val_f1_(macro): 0.20252489446037833


# 8/8 [==============================] - 3s 321ms/step - loss: 1.3089 - accuracy: 0.2110 - val_loss: 1.3646 - val_accuracy: 0.2100
# Epoch 11/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.29918032786885246
#  - val_f1_(macro): 0.10635952548743247


# 8/8 [==============================] - 1s 180ms/step - loss: 1.2487 - accuracy: 0.2749 - val_loss: 1.3623 - val_accuracy: 0.1450
# Epoch 12/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.2807377049180328
#  - val_f1_(macro): 0.0991582948345215


# 8/8 [==============================] - 2s 320ms/step - loss: 1.2545 - accuracy: 0.3325 - val_loss: 1.5100 - val_accuracy: 0.1225
# Epoch 13/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.40437158469945356
#  - val_f1_(macro): 0.2874708800856708


# 8/8 [==============================] - 2s 286ms/step - loss: 1.1472 - accuracy: 0.3431 - val_loss: 1.2558 - val_accuracy: 0.3525
# Epoch 14/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3176229508196721
#  - val_f1_(macro): 0.23380305843679822


# 8/8 [==============================] - 2s 221ms/step - loss: 0.9876 - accuracy: 0.3745 - val_loss: 1.3235 - val_accuracy: 0.2575
# Epoch 15/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.4733606557377049
#  - val_f1_(macro): 0.30382183900665316


# 8/8 [==============================] - 2s 254ms/step - loss: 1.0098 - accuracy: 0.4008 - val_loss: 1.1789 - val_accuracy: 0.4100
# Epoch 16/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.44672131147540983
#  - val_f1_(macro): 0.3143584506062942


# 8/8 [==============================] - 6s 634ms/step - loss: 0.8285 - accuracy: 0.4928 - val_loss: 1.2730 - val_accuracy: 0.3900
# Epoch 17/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4337431693989071
#  - val_f1_(macro): 0.3234142389990366


# 8/8 [==============================] - 3s 423ms/step - loss: 0.7475 - accuracy: 0.5404 - val_loss: 1.3126 - val_accuracy: 0.3825
# Epoch 18/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.4822404371584699
#  - val_f1_(macro): 0.3427354807950491


# 8/8 [==============================] - 3s 412ms/step - loss: 0.6852 - accuracy: 0.5523 - val_loss: 1.3041 - val_accuracy: 0.4175
# Epoch 19/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.39617486338797814
#  - val_f1_(macro): 0.2925404619674537


# 8/8 [==============================] - 3s 329ms/step - loss: 0.6348 - accuracy: 0.6068 - val_loss: 1.3653 - val_accuracy: 0.3775
# Epoch 20/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.3435792349726776
#  - val_f1_(macro): 0.2551887823451209


# 8/8 [==============================] - 1s 181ms/step - loss: 0.6347 - accuracy: 0.5823 - val_loss: 1.5100 - val_accuracy: 0.3425
# Epoch 21/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.44398907103825136
#  - val_f1_(macro): 0.29277788675098027


# 8/8 [==============================] - 2s 239ms/step - loss: 0.5674 - accuracy: 0.5942 - val_loss: 1.3974 - val_accuracy: 0.3900
# Epoch 22/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.430327868852459
#  - val_f1_(macro): 0.32488081062384466


# 8/8 [==============================] - 2s 212ms/step - loss: 0.4810 - accuracy: 0.6675 - val_loss: 1.5580 - val_accuracy: 0.4050
# Epoch 23/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.48770491803278687
#  - val_f1_(macro): 0.2935442702954484


# 8/8 [==============================] - 2s 216ms/step - loss: 0.4999 - accuracy: 0.6675 - val_loss: 1.5290 - val_accuracy: 0.4325
# Epoch 24/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4781420765027322
#  - val_f1_(macro): 0.3132041800351659


# 8/8 [==============================] - 1s 155ms/step - loss: 0.4865 - accuracy: 0.6963 - val_loss: 1.3934 - val_accuracy: 0.4250
# Epoch 25/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.47062841530054644
#  - val_f1_(macro): 0.3096748262146388


# 8/8 [==============================] - 2s 213ms/step - loss: 0.4648 - accuracy: 0.7044 - val_loss: 1.5890 - val_accuracy: 0.4150
# Epoch 26/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4371584699453552
#  - val_f1_(macro): 0.30752276911030285


# 8/8 [==============================] - 2s 193ms/step - loss: 0.4077 - accuracy: 0.7201 - val_loss: 1.7076 - val_accuracy: 0.4150
# CPU times: user 1min, sys: 1.85 s, total: 1min 1s
# Wall time: 1min 26s

# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     3     |     2     |    10     |    12     |
# -------------------------------------------------------------
# | Disagree  |     2     |     6     |    22     |    12     |
# -------------------------------------------------------------
# |  Discuss  |     8     |     5     |    84     |    98     |
# -------------------------------------------------------------
# | Unrelated |    11     |     4     |    48     |    73     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.12      0.11      0.12        27
#     Disagree       0.35      0.14      0.20        42
#      Discuss       0.51      0.43      0.47       195
#    Unrelated       0.37      0.54      0.44       136

#     accuracy                           0.41       400
#    macro avg       0.34      0.31      0.31       400
# weighted avg       0.42      0.41      0.41       400


In [35]:
# Execute #2:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 3s 8ms/step
#  - fnc_score: 0.5799180327868853
#  - val_f1_(macro): 0.1638655462184874


# 8/8 [==============================] - 31s 1s/step - loss: 1.3971 - accuracy: 0.3225 - val_loss: 1.3674 - val_accuracy: 0.4875
# Epoch 2/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5819672131147541
#  - val_f1_(macro): 0.16779104917791046


# 8/8 [==============================] - 2s 268ms/step - loss: 1.3887 - accuracy: 0.2135 - val_loss: 1.3796 - val_accuracy: 0.4900
# Epoch 3/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.26639344262295084
#  - val_f1_(macro): 0.04751131221719457


# 8/8 [==============================] - 2s 243ms/step - loss: 1.3869 - accuracy: 0.3356 - val_loss: 1.3967 - val_accuracy: 0.1050
# Epoch 4/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.23565573770491804
#  - val_f1_(macro): 0.03161592505854802


# 8/8 [==============================] - 2s 272ms/step - loss: 1.3863 - accuracy: 0.1252 - val_loss: 1.4050 - val_accuracy: 0.0675
# Epoch 5/70
# 13/13 [==============================] - 0s 13ms/step
#  - fnc_score: 0.26639344262295084
#  - val_f1_(macro): 0.04751131221719457


# 8/8 [==============================] - 2s 276ms/step - loss: 1.3802 - accuracy: 0.1240 - val_loss: 1.3751 - val_accuracy: 0.1050
# Epoch 6/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.27049180327868855
#  - val_f1_(macro): 0.05403664633304231


# 8/8 [==============================] - 3s 348ms/step - loss: 1.3712 - accuracy: 0.1152 - val_loss: 1.3514 - val_accuracy: 0.1100
# Epoch 7/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.27527322404371585
#  - val_f1_(macro): 0.06875048073464986


# 8/8 [==============================] - 3s 365ms/step - loss: 1.3871 - accuracy: 0.1353 - val_loss: 1.3952 - val_accuracy: 0.1200
# Epoch 8/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.26844262295081966
#  - val_f1_(macro): 0.06649758262661487


# 8/8 [==============================] - 2s 222ms/step - loss: 1.3708 - accuracy: 0.3143 - val_loss: 1.5566 - val_accuracy: 0.1075
# Epoch 9/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.38866120218579236
#  - val_f1_(macro): 0.22789298285697734


# 8/8 [==============================] - 2s 275ms/step - loss: 1.3457 - accuracy: 0.2536 - val_loss: 1.3484 - val_accuracy: 0.2625
# Epoch 10/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.2957650273224044
#  - val_f1_(macro): 0.2179976338803435


# 8/8 [==============================] - 2s 240ms/step - loss: 1.2911 - accuracy: 0.3068 - val_loss: 1.2738 - val_accuracy: 0.2900
# Epoch 11/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.27049180327868855
#  - val_f1_(macro): 0.06786403085824494


# 8/8 [==============================] - 2s 186ms/step - loss: 1.2277 - accuracy: 0.1672 - val_loss: 1.4066 - val_accuracy: 0.1100
# Epoch 12/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.2855191256830601
#  - val_f1_(macro): 0.11901595951482172


# 8/8 [==============================] - 2s 329ms/step - loss: 1.1490 - accuracy: 0.1534 - val_loss: 1.4470 - val_accuracy: 0.1350
# Epoch 13/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.33060109289617484
#  - val_f1_(macro): 0.1795568228768955


# 8/8 [==============================] - 2s 302ms/step - loss: 1.0420 - accuracy: 0.2010 - val_loss: 1.3961 - val_accuracy: 0.2000
# Epoch 14/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5
#  - val_f1_(macro): 0.2684038711284268


# 8/8 [==============================] - 2s 256ms/step - loss: 0.9969 - accuracy: 0.3832 - val_loss: 1.2961 - val_accuracy: 0.3900
# Epoch 15/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.366120218579235
#  - val_f1_(macro): 0.30702911968029656


# 8/8 [==============================] - 2s 209ms/step - loss: 0.8788 - accuracy: 0.5172 - val_loss: 1.2621 - val_accuracy: 0.3600
# Epoch 16/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.28825136612021857
#  - val_f1_(macro): 0.2084533704206541


# 8/8 [==============================] - 2s 240ms/step - loss: 0.8793 - accuracy: 0.4051 - val_loss: 1.5727 - val_accuracy: 0.2225
# Epoch 17/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3722677595628415
#  - val_f1_(macro): 0.2933873373359027


# 8/8 [==============================] - 2s 236ms/step - loss: 0.8664 - accuracy: 0.4728 - val_loss: 1.4508 - val_accuracy: 0.2950
# Epoch 18/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.3825136612021858
#  - val_f1_(macro): 0.32948130068982273


# 8/8 [==============================] - 2s 218ms/step - loss: 0.7359 - accuracy: 0.4421 - val_loss: 1.2862 - val_accuracy: 0.3625
# Epoch 19/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.5532786885245902
#  - val_f1_(macro): 0.3099035676966711


# 8/8 [==============================] - 4s 452ms/step - loss: 0.6657 - accuracy: 0.4602 - val_loss: 1.2191 - val_accuracy: 0.4550
# Epoch 20/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3831967213114754
#  - val_f1_(macro): 0.3259067786965149


# 8/8 [==============================] - 2s 219ms/step - loss: 0.5983 - accuracy: 0.5059 - val_loss: 1.2399 - val_accuracy: 0.4025
# Epoch 21/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.48975409836065575
#  - val_f1_(macro): 0.3889127183825497


# 8/8 [==============================] - 2s 214ms/step - loss: 0.5634 - accuracy: 0.5567 - val_loss: 1.2753 - val_accuracy: 0.4625
# Epoch 22/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5348360655737705
#  - val_f1_(macro): 0.4026019464716432


# 8/8 [==============================] - 2s 248ms/step - loss: 0.5425 - accuracy: 0.5254 - val_loss: 1.3023 - val_accuracy: 0.4775
# Epoch 23/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5102459016393442
#  - val_f1_(macro): 0.3657465066835307


# 8/8 [==============================] - 2s 186ms/step - loss: 0.5510 - accuracy: 0.5611 - val_loss: 1.2933 - val_accuracy: 0.4325
# Epoch 24/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.4296448087431694
#  - val_f1_(macro): 0.3835136558093102


# 8/8 [==============================] - 1s 96ms/step - loss: 0.5314 - accuracy: 0.5780 - val_loss: 1.2223 - val_accuracy: 0.4250
# Epoch 25/70
# 13/13 [==============================] - 0s 6ms/step
#  - fnc_score: 0.5191256830601093
#  - val_f1_(macro): 0.4355796183719543


# 8/8 [==============================] - 1s 199ms/step - loss: 0.4358 - accuracy: 0.6243 - val_loss: 1.3955 - val_accuracy: 0.4925
# Epoch 26/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.4173497267759563
#  - val_f1_(macro): 0.3771519334684229


# 8/8 [==============================] - 1s 127ms/step - loss: 0.4583 - accuracy: 0.5986 - val_loss: 1.4114 - val_accuracy: 0.4125
# Epoch 27/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.4760928961748634
#  - val_f1_(macro): 0.40441148817019584


# 8/8 [==============================] - 3s 343ms/step - loss: 0.4269 - accuracy: 0.6068 - val_loss: 1.3455 - val_accuracy: 0.4425
# Epoch 28/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.5546448087431693
#  - val_f1_(macro): 0.3978243021346469


# 8/8 [==============================] - 2s 218ms/step - loss: 0.4005 - accuracy: 0.6431 - val_loss: 1.4628 - val_accuracy: 0.5150
# Epoch 29/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.546448087431694
#  - val_f1_(macro): 0.41044089476145224


# 8/8 [==============================] - 1s 97ms/step - loss: 0.4198 - accuracy: 0.6556 - val_loss: 1.4089 - val_accuracy: 0.5000
# Epoch 30/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5443989071038251
#  - val_f1_(macro): 0.35552639681967757


# 8/8 [==============================] - 2s 184ms/step - loss: 0.4004 - accuracy: 0.6625 - val_loss: 1.4711 - val_accuracy: 0.4725
# Epoch 31/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.48565573770491804
#  - val_f1_(macro): 0.3873791526014608


# 8/8 [==============================] - 1s 176ms/step - loss: 0.4309 - accuracy: 0.6518 - val_loss: 1.3769 - val_accuracy: 0.4625
# Epoch 32/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3251366120218579
#  - val_f1_(macro): 0.27907322628670106


# 8/8 [==============================] - 2s 244ms/step - loss: 0.4121 - accuracy: 0.6587 - val_loss: 1.5278 - val_accuracy: 0.3600
# Epoch 33/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.41939890710382516
#  - val_f1_(macro): 0.3796576220489264


# 8/8 [==============================] - 2s 220ms/step - loss: 0.3995 - accuracy: 0.6957 - val_loss: 1.3689 - val_accuracy: 0.4125
# Epoch 34/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.4931693989071038
#  - val_f1_(macro): 0.39781850956494


# 8/8 [==============================] - 1s 121ms/step - loss: 0.3502 - accuracy: 0.7019 - val_loss: 1.4690 - val_accuracy: 0.4800
# Epoch 35/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.47882513661202186
#  - val_f1_(macro): 0.42800334168755216


# 8/8 [==============================] - 1s 158ms/step - loss: 0.3086 - accuracy: 0.7671 - val_loss: 1.5771 - val_accuracy: 0.4625
# Epoch 36/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5423497267759563
#  - val_f1_(macro): 0.4118887450787989


# 8/8 [==============================] - 2s 187ms/step - loss: 0.3269 - accuracy: 0.7570 - val_loss: 1.6210 - val_accuracy: 0.4950
# Epoch 37/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5150273224043715
#  - val_f1_(macro): 0.43029829777755224


# 8/8 [==============================] - 2s 184ms/step - loss: 0.3608 - accuracy: 0.7633 - val_loss: 1.2796 - val_accuracy: 0.4875
# Epoch 38/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5437158469945356
#  - val_f1_(macro): 0.4113260082836241


# 8/8 [==============================] - 1s 98ms/step - loss: 0.3230 - accuracy: 0.7545 - val_loss: 1.4083 - val_accuracy: 0.4775
# Epoch 39/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5416666666666666
#  - val_f1_(macro): 0.4381889634405004


# 8/8 [==============================] - 1s 174ms/step - loss: 0.3025 - accuracy: 0.7827 - val_loss: 1.4926 - val_accuracy: 0.5025
# Epoch 40/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.5122950819672131
#  - val_f1_(macro): 0.40632648044862063


# 8/8 [==============================] - 1s 142ms/step - loss: 0.2949 - accuracy: 0.8071 - val_loss: 1.5008 - val_accuracy: 0.5025
# Epoch 41/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5450819672131147
#  - val_f1_(macro): 0.45307844147859744


# 8/8 [==============================] - 1s 154ms/step - loss: 0.3063 - accuracy: 0.8121 - val_loss: 1.5143 - val_accuracy: 0.5250
# Epoch 42/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5450819672131147
#  - val_f1_(macro): 0.45590633417722687


# 8/8 [==============================] - 2s 210ms/step - loss: 0.2410 - accuracy: 0.8328 - val_loss: 1.5382 - val_accuracy: 0.5225
# Epoch 43/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5580601092896175
#  - val_f1_(macro): 0.4256320997427271


# 8/8 [==============================] - 1s 125ms/step - loss: 0.2174 - accuracy: 0.8585 - val_loss: 1.6600 - val_accuracy: 0.5450
# Epoch 44/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.476775956284153
#  - val_f1_(macro): 0.4200965891264399


# 8/8 [==============================] - 2s 239ms/step - loss: 0.2019 - accuracy: 0.8785 - val_loss: 1.6982 - val_accuracy: 0.4575
# CPU times: user 1min 17s, sys: 2.38 s, total: 1min 19s
# Wall time: 1min 45s

# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     8     |     0     |     7     |    12     |
# -------------------------------------------------------------
# | Disagree  |     2     |    15     |    19     |     6     |
# -------------------------------------------------------------
# |  Discuss  |    19     |     1     |    78     |    97     |
# -------------------------------------------------------------
# | Unrelated |    16     |     2     |    36     |    82     |
# -------------------------------------------------------------

#               precision    recall  f1-score   support

#        Agree       0.18      0.30      0.22        27
#     Disagree       0.83      0.36      0.50        42
#      Discuss       0.56      0.40      0.47       195
#    Unrelated       0.42      0.60      0.49       136

#     accuracy                           0.46       400
#    macro avg       0.50      0.41      0.42       400
# weighted avg       0.51      0.46      0.46       400



In [36]:
# Execute #3:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 2s 9ms/step
#  - fnc_score: 0.2786885245901639
#  - val_f1_(macro): 0.12686567164179105


# 8/8 [==============================] - 29s 922ms/step - loss: 1.3928 - accuracy: 0.2235 - val_loss: 1.3528 - val_accuracy: 0.3400
# Epoch 2/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.23497267759562843
#  - val_f1_(macro): 0.03919320244184128


# 8/8 [==============================] - 3s 343ms/step - loss: 1.3874 - accuracy: 0.1691 - val_loss: 1.3871 - val_accuracy: 0.0725
# Epoch 3/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.26639344262295084
#  - val_f1_(macro): 0.07512829284981183


# 8/8 [==============================] - 3s 372ms/step - loss: 1.3857 - accuracy: 0.2198 - val_loss: 1.3961 - val_accuracy: 0.1050
# Epoch 4/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.2937158469945355
#  - val_f1_(macro): 0.16906872367398684


# 8/8 [==============================] - 2s 277ms/step - loss: 1.3817 - accuracy: 0.1146 - val_loss: 1.3636 - val_accuracy: 0.3325
# Epoch 5/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.34153005464480873
#  - val_f1_(macro): 0.18325754054001647


# 8/8 [==============================] - 2s 249ms/step - loss: 1.3615 - accuracy: 0.1885 - val_loss: 1.3735 - val_accuracy: 0.1975
# Epoch 6/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.23497267759562843
#  - val_f1_(macro): 0.03169014084507042


# 8/8 [==============================] - 2s 205ms/step - loss: 1.3977 - accuracy: 0.1716 - val_loss: 1.4410 - val_accuracy: 0.0675
# Epoch 7/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.27049180327868855
#  - val_f1_(macro): 0.15545151221612313


# 8/8 [==============================] - 2s 244ms/step - loss: 1.3736 - accuracy: 0.1885 - val_loss: 1.3557 - val_accuracy: 0.3075
# Epoch 8/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.30396174863387976
#  - val_f1_(macro): 0.19512758000006294


# 8/8 [==============================] - 2s 212ms/step - loss: 1.3439 - accuracy: 0.2630 - val_loss: 1.3417 - val_accuracy: 0.2475
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.3087431693989071
#  - val_f1_(macro): 0.18986279802069278


# 8/8 [==============================] - 3s 416ms/step - loss: 1.2450 - accuracy: 0.2893 - val_loss: 1.3884 - val_accuracy: 0.2075
# Epoch 10/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.31420765027322406
#  - val_f1_(macro): 0.21451924306354686


# 8/8 [==============================] - 1s 204ms/step - loss: 1.1900 - accuracy: 0.3475 - val_loss: 1.4665 - val_accuracy: 0.2200
# Epoch 11/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3237704918032787
#  - val_f1_(macro): 0.1764633493846977


# 8/8 [==============================] - 1s 122ms/step - loss: 1.1850 - accuracy: 0.3932 - val_loss: 1.4019 - val_accuracy: 0.1750
# Epoch 12/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.37636612021857924
#  - val_f1_(macro): 0.26993904301596605


# 8/8 [==============================] - 2s 209ms/step - loss: 1.1440 - accuracy: 0.2924 - val_loss: 1.4127 - val_accuracy: 0.2775
# Epoch 13/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.3777322404371585
#  - val_f1_(macro): 0.3072482589630097


# 8/8 [==============================] - 2s 246ms/step - loss: 0.9680 - accuracy: 0.4170 - val_loss: 1.4341 - val_accuracy: 0.3100
# Epoch 14/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.31010928961748635
#  - val_f1_(macro): 0.2159696109313946


# 8/8 [==============================] - 2s 214ms/step - loss: 0.8781 - accuracy: 0.4483 - val_loss: 1.2835 - val_accuracy: 0.3200
# Epoch 15/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.48975409836065575
#  - val_f1_(macro): 0.3191715085287847


# 8/8 [==============================] - 2s 245ms/step - loss: 0.8979 - accuracy: 0.4584 - val_loss: 1.1351 - val_accuracy: 0.4425
# Epoch 16/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.35655737704918034
#  - val_f1_(macro): 0.3229855776070565


# 8/8 [==============================] - 2s 262ms/step - loss: 0.7563 - accuracy: 0.5404 - val_loss: 1.2953 - val_accuracy: 0.3625
# Epoch 17/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4378415300546448
#  - val_f1_(macro): 0.26849808011247467


# 8/8 [==============================] - 3s 310ms/step - loss: 0.6925 - accuracy: 0.5679 - val_loss: 1.7874 - val_accuracy: 0.3275
# Epoch 18/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4883879781420765
#  - val_f1_(macro): 0.2755482951058531


# 8/8 [==============================] - 2s 219ms/step - loss: 0.6739 - accuracy: 0.5216 - val_loss: 1.4575 - val_accuracy: 0.3800
# Epoch 19/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.40437158469945356
#  - val_f1_(macro): 0.3351506955120169


# 8/8 [==============================] - 2s 217ms/step - loss: 0.6373 - accuracy: 0.5435 - val_loss: 1.4691 - val_accuracy: 0.3450
# Epoch 20/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.43510928961748635
#  - val_f1_(macro): 0.36320068265260663


# 8/8 [==============================] - 1s 152ms/step - loss: 0.5398 - accuracy: 0.6205 - val_loss: 1.3494 - val_accuracy: 0.4075
# Epoch 21/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.4296448087431694
#  - val_f1_(macro): 0.37367743352941774


# 8/8 [==============================] - 2s 230ms/step - loss: 0.4696 - accuracy: 0.6612 - val_loss: 1.3683 - val_accuracy: 0.4100
# Epoch 22/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.4139344262295082
#  - val_f1_(macro): 0.3540023626501614


# 8/8 [==============================] - 1s 192ms/step - loss: 0.4117 - accuracy: 0.6869 - val_loss: 1.4978 - val_accuracy: 0.3925
# Epoch 23/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.3524590163934426
#  - val_f1_(macro): 0.32289901893639006


# 8/8 [==============================] - 1s 205ms/step - loss: 0.4368 - accuracy: 0.6988 - val_loss: 1.5928 - val_accuracy: 0.3325
# CPU times: user 52.5 s, sys: 1.64 s, total: 54.1 s
# Wall time: 1min 12s

# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     9     |     0     |     8     |    10     |
# -------------------------------------------------------------
# | Disagree  |     6     |    20     |     1     |    15     |
# -------------------------------------------------------------
# |  Discuss  |    18     |    20     |    35     |    122    |
# -------------------------------------------------------------
# | Unrelated |    18     |    15     |    34     |    69     |
# -------------------------------------------------------------



#               precision    recall  f1-score   support

#        Agree       0.18      0.33      0.23        27
#     Disagree       0.36      0.48      0.41        42
#      Discuss       0.45      0.18      0.26       195
#    Unrelated       0.32      0.51      0.39       136

#     accuracy                           0.33       400
#    macro avg       0.33      0.37      0.32       400
# weighted avg       0.38      0.33      0.32       400


In [37]:
# Execute #4:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 2s 8ms/step
#  - fnc_score: 0.26639344262295084
#  - val_f1_(macro): 0.04751131221719457


# 8/8 [==============================] - 30s 1s/step - loss: 1.3905 - accuracy: 0.1121 - val_loss: 1.3737 - val_accuracy: 0.1050
# Epoch 2/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3025956284153005
#  - val_f1_(macro): 0.16513517737146433


# 8/8 [==============================] - 2s 268ms/step - loss: 1.3926 - accuracy: 0.3832 - val_loss: 1.3904 - val_accuracy: 0.2875
# Epoch 3/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2650273224043716
#  - val_f1_(macro): 0.06972188995215312


# 8/8 [==============================] - 2s 271ms/step - loss: 1.3868 - accuracy: 0.1421 - val_loss: 1.3849 - val_accuracy: 0.1175
# Epoch 4/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.2657103825136612
#  - val_f1_(macro): 0.12736231579579219


# 8/8 [==============================] - 3s 352ms/step - loss: 1.3852 - accuracy: 0.1152 - val_loss: 1.3802 - val_accuracy: 0.1525
# Epoch 5/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2527322404371585
#  - val_f1_(macro): 0.10360825173602825


# 8/8 [==============================] - 3s 335ms/step - loss: 1.3837 - accuracy: 0.3901 - val_loss: 1.3719 - val_accuracy: 0.1025
# Epoch 6/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3005464480874317
#  - val_f1_(macro): 0.15720493411830672


# 8/8 [==============================] - 2s 242ms/step - loss: 1.3682 - accuracy: 0.2060 - val_loss: 1.4068 - val_accuracy: 0.2025
# Epoch 7/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.23975409836065573
#  - val_f1_(macro): 0.04493839493839494


# 8/8 [==============================] - 2s 245ms/step - loss: 1.3684 - accuracy: 0.2448 - val_loss: 1.3774 - val_accuracy: 0.0750
# Epoch 8/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.30601092896174864
#  - val_f1_(macro): 0.2229366502141926


# 8/8 [==============================] - 2s 247ms/step - loss: 1.3369 - accuracy: 0.1578 - val_loss: 1.3540 - val_accuracy: 0.2650
# Epoch 9/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.38866120218579236
#  - val_f1_(macro): 0.2527324423668338


# 8/8 [==============================] - 2s 280ms/step - loss: 1.3053 - accuracy: 0.2793 - val_loss: 1.3826 - val_accuracy: 0.2800
# Epoch 10/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.31693989071038253
#  - val_f1_(macro): 0.22547910714649136


# 8/8 [==============================] - 2s 275ms/step - loss: 1.2045 - accuracy: 0.3168 - val_loss: 1.3251 - val_accuracy: 0.2875
# Epoch 11/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.4173497267759563
#  - val_f1_(macro): 0.2847682551281382


# 8/8 [==============================] - 3s 358ms/step - loss: 1.1341 - accuracy: 0.2799 - val_loss: 1.2280 - val_accuracy: 0.4175
# Epoch 12/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3005464480874317
#  - val_f1_(macro): 0.2072711555387904


# 8/8 [==============================] - 2s 245ms/step - loss: 1.1101 - accuracy: 0.3137 - val_loss: 1.3947 - val_accuracy: 0.1975
# Epoch 13/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.30327868852459017
#  - val_f1_(macro): 0.16522052704576976


# 8/8 [==============================] - 2s 272ms/step - loss: 1.0271 - accuracy: 0.3231 - val_loss: 1.5089 - val_accuracy: 0.1500
# Epoch 14/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.3476775956284153
#  - val_f1_(macro): 0.21876060056021784


# 8/8 [==============================] - 1s 205ms/step - loss: 0.9395 - accuracy: 0.3669 - val_loss: 1.7056 - val_accuracy: 0.2150
# Epoch 15/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3360655737704918
#  - val_f1_(macro): 0.2602959966041306


# 8/8 [==============================] - 2s 229ms/step - loss: 0.9229 - accuracy: 0.3914 - val_loss: 1.3795 - val_accuracy: 0.2575
# Epoch 16/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.40915300546448086
#  - val_f1_(macro): 0.3295677214483149


# 8/8 [==============================] - 1s 184ms/step - loss: 0.8445 - accuracy: 0.3795 - val_loss: 1.3406 - val_accuracy: 0.3700
# Epoch 17/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.3831967213114754
#  - val_f1_(macro): 0.3031663774000939


# 8/8 [==============================] - 2s 298ms/step - loss: 0.7739 - accuracy: 0.5034 - val_loss: 1.3619 - val_accuracy: 0.3700
# Epoch 18/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.43989071038251365
#  - val_f1_(macro): 0.3363376312666373


# 8/8 [==============================] - 2s 294ms/step - loss: 0.7341 - accuracy: 0.4208 - val_loss: 1.3584 - val_accuracy: 0.3825
# Epoch 19/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.5266393442622951
#  - val_f1_(macro): 0.3389380858212317


# 8/8 [==============================] - 2s 263ms/step - loss: 0.7054 - accuracy: 0.5128 - val_loss: 1.2267 - val_accuracy: 0.4550
# Epoch 20/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.28825136612021857
#  - val_f1_(macro): 0.15797831363827963


# 8/8 [==============================] - 2s 219ms/step - loss: 0.7291 - accuracy: 0.4559 - val_loss: 1.5515 - val_accuracy: 0.3275
# Epoch 21/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3900273224043716
#  - val_f1_(macro): 0.23892891382810738


# 8/8 [==============================] - 2s 211ms/step - loss: 0.8802 - accuracy: 0.4076 - val_loss: 1.4422 - val_accuracy: 0.2725
# Epoch 22/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.33811475409836067
#  - val_f1_(macro): 0.2608554322280943


# 8/8 [==============================] - 1s 207ms/step - loss: 0.7251 - accuracy: 0.5229 - val_loss: 1.5222 - val_accuracy: 0.2750
# Epoch 23/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.453551912568306
#  - val_f1_(macro): 0.25460086299892126


# 8/8 [==============================] - 1s 155ms/step - loss: 0.6281 - accuracy: 0.4502 - val_loss: 1.5373 - val_accuracy: 0.3400
# Epoch 24/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.36543715846994534
#  - val_f1_(macro): 0.2909946569721006


# 8/8 [==============================] - 1s 153ms/step - loss: 0.5767 - accuracy: 0.5548 - val_loss: 1.4263 - val_accuracy: 0.3600
# Epoch 25/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.42418032786885246
#  - val_f1_(macro): 0.3172069042100328


# 8/8 [==============================] - 1s 156ms/step - loss: 0.5210 - accuracy: 0.5823 - val_loss: 1.3812 - val_accuracy: 0.4125
# Epoch 26/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.5027322404371585
#  - val_f1_(macro): 0.3634466887520172


# 8/8 [==============================] - 2s 185ms/step - loss: 0.5268 - accuracy: 0.5729 - val_loss: 1.3684 - val_accuracy: 0.4550
# Epoch 27/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.49658469945355194
#  - val_f1_(macro): 0.3432172470978441


# 8/8 [==============================] - 2s 332ms/step - loss: 0.4382 - accuracy: 0.6155 - val_loss: 1.4667 - val_accuracy: 0.4500
# Epoch 28/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5286885245901639
#  - val_f1_(macro): 0.346781660692951


# 8/8 [==============================] - 2s 187ms/step - loss: 0.3963 - accuracy: 0.6951 - val_loss: 1.6147 - val_accuracy: 0.4725
# Epoch 29/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5239071038251366
#  - val_f1_(macro): 0.3678910634171111


# 8/8 [==============================] - 2s 187ms/step - loss: 0.3956 - accuracy: 0.7057 - val_loss: 1.6512 - val_accuracy: 0.4725
# Epoch 30/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.48155737704918034
#  - val_f1_(macro): 0.3237038378347199


# 8/8 [==============================] - 2s 220ms/step - loss: 0.3847 - accuracy: 0.7157 - val_loss: 1.6989 - val_accuracy: 0.4050
# Epoch 31/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.4733606557377049
#  - val_f1_(macro): 0.35077720221792025


# 8/8 [==============================] - 2s 219ms/step - loss: 0.3673 - accuracy: 0.7633 - val_loss: 1.6372 - val_accuracy: 0.4175
# Epoch 32/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5382513661202186
#  - val_f1_(macro): 0.34506874621153677


# 8/8 [==============================] - 2s 183ms/step - loss: 0.3366 - accuracy: 0.7752 - val_loss: 1.6031 - val_accuracy: 0.4875
# Epoch 33/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5006830601092896
#  - val_f1_(macro): 0.3535846586857119


# 8/8 [==============================] - 1s 132ms/step - loss: 0.3115 - accuracy: 0.7871 - val_loss: 1.6660 - val_accuracy: 0.4500
# Epoch 34/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.5163934426229508
#  - val_f1_(macro): 0.3316121244498127


# 8/8 [==============================] - 1s 160ms/step - loss: 0.3070 - accuracy: 0.7909 - val_loss: 1.8256 - val_accuracy: 0.4450
# Epoch 35/70
# 13/13 [==============================] - 0s 12ms/step
#  - fnc_score: 0.519808743169399
#  - val_f1_(macro): 0.33262854691426125


# 8/8 [==============================] - 2s 351ms/step - loss: 0.2735 - accuracy: 0.8303 - val_loss: 1.8312 - val_accuracy: 0.4550
# Epoch 36/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5382513661202186
#  - val_f1_(macro): 0.35820793788963123


# 8/8 [==============================] - 1s 112ms/step - loss: 0.2397 - accuracy: 0.8566 - val_loss: 1.8147 - val_accuracy: 0.4900
# Epoch 37/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.4952185792349727
#  - val_f1_(macro): 0.34626232639303445


# 8/8 [==============================] - 1s 148ms/step - loss: 0.2378 - accuracy: 0.8516 - val_loss: 1.8555 - val_accuracy: 0.4400
# Epoch 38/70
# 13/13 [==============================] - 0s 6ms/step
#  - fnc_score: 0.5239071038251366
#  - val_f1_(macro): 0.3619605058753453


# 8/8 [==============================] - 1s 171ms/step - loss: 0.2577 - accuracy: 0.8566 - val_loss: 1.7721 - val_accuracy: 0.4700
# Epoch 39/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5423497267759563
#  - val_f1_(macro): 0.3526135075609002


# 8/8 [==============================] - 1s 152ms/step - loss: 0.2026 - accuracy: 0.8723 - val_loss: 1.9596 - val_accuracy: 0.4800
# Epoch 40/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5314207650273224
#  - val_f1_(macro): 0.35293831742665255


# 8/8 [==============================] - 1s 147ms/step - loss: 0.2293 - accuracy: 0.8785 - val_loss: 1.8860 - val_accuracy: 0.4750
# Epoch 41/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5273224043715847
#  - val_f1_(macro): 0.3706871315141752


# 8/8 [==============================] - 1s 123ms/step - loss: 0.1787 - accuracy: 0.8992 - val_loss: 1.9044 - val_accuracy: 0.4825
# Epoch 42/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5266393442622951
#  - val_f1_(macro): 0.3417190472052448


# 8/8 [==============================] - 1s 150ms/step - loss: 0.1944 - accuracy: 0.8867 - val_loss: 1.8494 - val_accuracy: 0.4675
# Epoch 43/70
# 13/13 [==============================] - 0s 6ms/step
#  - fnc_score: 0.5505464480874317
#  - val_f1_(macro): 0.3726680951067821


# 8/8 [==============================] - 1s 115ms/step - loss: 0.1543 - accuracy: 0.9123 - val_loss: 1.8798 - val_accuracy: 0.5100
# Epoch 44/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5273224043715847
#  - val_f1_(macro): 0.3521875702127046


# 8/8 [==============================] - 1s 153ms/step - loss: 0.1809 - accuracy: 0.9029 - val_loss: 2.0172 - val_accuracy: 0.4700
# Epoch 45/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5635245901639344
#  - val_f1_(macro): 0.3348357887510312


# 8/8 [==============================] - 2s 221ms/step - loss: 0.1613 - accuracy: 0.8979 - val_loss: 1.8971 - val_accuracy: 0.4875
# CPU times: user 1min 17s, sys: 2.46 s, total: 1min 19s
# Wall time: 1min 45s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     3     |     1     |    19     |     4     |
# -------------------------------------------------------------
# | Disagree  |     2     |    11     |    28     |     1     |
# -------------------------------------------------------------
# |  Discuss  |     9     |     9     |    158    |    19     |
# -------------------------------------------------------------
# | Unrelated |     6     |     3     |    104    |    23     |
# -------------------------------------------------------------


#               precision    recall  f1-score   support

#        Agree       0.15      0.11      0.13        27
#     Disagree       0.46      0.26      0.33        42
#      Discuss       0.51      0.81      0.63       195
#    Unrelated       0.49      0.17      0.25       136

#     accuracy                           0.49       400
#    macro avg       0.40      0.34      0.33       400
# weighted avg       0.47      0.49      0.43       400



In [38]:
# Execute 5:

# WARNING:absl:`lr` is deprecated in Keras optimizer, please use `learning_rate` or use the legacy optimizer, e.g.,tf.keras.optimizers.legacy.Adam.
# Used optimizer: Adam, lr=0.001
# Epoch 1/70
# 13/13 [==============================] - 3s 9ms/step
#  - fnc_score: 0.26639344262295084
#  - val_f1_(macro): 0.04751131221719457


# 8/8 [==============================] - 23s 1s/step - loss: 1.3937 - accuracy: 0.1947 - val_loss: 1.3962 - val_accuracy: 0.1050
# Epoch 2/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5300546448087432
#  - val_f1_(macro): 0.19689187758127713


# 8/8 [==============================] - 2s 247ms/step - loss: 1.3892 - accuracy: 0.1503 - val_loss: 1.3758 - val_accuracy: 0.4300
# Epoch 3/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.26707650273224043
#  - val_f1_(macro): 0.05143311319054096


# 8/8 [==============================] - 2s 253ms/step - loss: 1.3861 - accuracy: 0.3876 - val_loss: 1.3922 - val_accuracy: 0.1075
# Epoch 4/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2786885245901639
#  - val_f1_(macro): 0.12686567164179105


# 8/8 [==============================] - 2s 248ms/step - loss: 1.3861 - accuracy: 0.2348 - val_loss: 1.3669 - val_accuracy: 0.3400
# Epoch 5/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2814207650273224
#  - val_f1_(macro): 0.15113750418200067


# 8/8 [==============================] - 2s 273ms/step - loss: 1.3820 - accuracy: 0.2267 - val_loss: 1.3568 - val_accuracy: 0.2875
# Epoch 6/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.2937158469945355
#  - val_f1_(macro): 0.16776371020211134


# 8/8 [==============================] - 3s 396ms/step - loss: 1.3746 - accuracy: 0.2204 - val_loss: 1.3267 - val_accuracy: 0.2925
# Epoch 7/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2616120218579235
#  - val_f1_(macro): 0.08090277777777777


# 8/8 [==============================] - 2s 268ms/step - loss: 1.3586 - accuracy: 0.2517 - val_loss: 1.7128 - val_accuracy: 0.1000
# Epoch 8/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3073770491803279
#  - val_f1_(macro): 0.1808212058212058


# 8/8 [==============================] - 2s 221ms/step - loss: 1.3370 - accuracy: 0.2392 - val_loss: 1.3047 - val_accuracy: 0.2800
# Epoch 9/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.34153005464480873
#  - val_f1_(macro): 0.19108873582557795


# 8/8 [==============================] - 2s 223ms/step - loss: 1.2506 - accuracy: 0.3256 - val_loss: 1.4418 - val_accuracy: 0.2050
# Epoch 10/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.2773224043715847
#  - val_f1_(macro): 0.15715679446692524


# 8/8 [==============================] - 2s 246ms/step - loss: 1.2314 - accuracy: 0.2912 - val_loss: 1.3308 - val_accuracy: 0.2250
# Epoch 11/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3360655737704918
#  - val_f1_(macro): 0.23001325910337284


# 8/8 [==============================] - 2s 213ms/step - loss: 1.1152 - accuracy: 0.3882 - val_loss: 1.4072 - val_accuracy: 0.2525
# Epoch 12/70
# 13/13 [==============================] - 0s 9ms/step
#  - fnc_score: 0.34153005464480873
#  - val_f1_(macro): 0.307052584066802


# 8/8 [==============================] - 2s 223ms/step - loss: 1.0035 - accuracy: 0.3663 - val_loss: 1.2022 - val_accuracy: 0.3700
# Epoch 13/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.4487704918032787
#  - val_f1_(macro): 0.31891873583604297


# 8/8 [==============================] - 3s 397ms/step - loss: 0.9132 - accuracy: 0.4452 - val_loss: 1.2325 - val_accuracy: 0.3850
# Epoch 14/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.3743169398907104
#  - val_f1_(macro): 0.3085974454395507


# 8/8 [==============================] - 2s 216ms/step - loss: 0.8578 - accuracy: 0.4584 - val_loss: 1.2732 - val_accuracy: 0.3500
# Epoch 15/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5778688524590164
#  - val_f1_(macro): 0.32611137416993075


# 8/8 [==============================] - 1s 148ms/step - loss: 0.8163 - accuracy: 0.4070 - val_loss: 1.2421 - val_accuracy: 0.4900
# Epoch 16/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.42213114754098363
#  - val_f1_(macro): 0.3139594188930052


# 8/8 [==============================] - 2s 185ms/step - loss: 0.7148 - accuracy: 0.5805 - val_loss: 1.2853 - val_accuracy: 0.3650
# Epoch 17/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5539617486338798
#  - val_f1_(macro): 0.336160397704698


# 8/8 [==============================] - 2s 185ms/step - loss: 0.6805 - accuracy: 0.4909 - val_loss: 1.3314 - val_accuracy: 0.4750
# Epoch 18/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.38729508196721313
#  - val_f1_(macro): 0.3291324810262832


# 8/8 [==============================] - 2s 211ms/step - loss: 0.6284 - accuracy: 0.5711 - val_loss: 1.3266 - val_accuracy: 0.3625
# Epoch 19/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.5095628415300546
#  - val_f1_(macro): 0.29405600819132016


# 8/8 [==============================] - 2s 245ms/step - loss: 0.6276 - accuracy: 0.4984 - val_loss: 1.4578 - val_accuracy: 0.4100
# Epoch 20/70
# 13/13 [==============================] - 0s 10ms/step
#  - fnc_score: 0.3825136612021858
#  - val_f1_(macro): 0.3334356506771367


# 8/8 [==============================] - 2s 232ms/step - loss: 0.6213 - accuracy: 0.5341 - val_loss: 1.4636 - val_accuracy: 0.3750
# Epoch 21/70
# 13/13 [==============================] - 0s 11ms/step
#  - fnc_score: 0.3818306010928962
#  - val_f1_(macro): 0.34317404428881576


# 8/8 [==============================] - 3s 363ms/step - loss: 0.5895 - accuracy: 0.5742 - val_loss: 1.3665 - val_accuracy: 0.3775
# Epoch 22/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.4603825136612022
#  - val_f1_(macro): 0.3943570353097548


# 8/8 [==============================] - 1s 184ms/step - loss: 0.4798 - accuracy: 0.5654 - val_loss: 1.4324 - val_accuracy: 0.4250
# Epoch 23/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5116120218579235
#  - val_f1_(macro): 0.3210088807906449


# 8/8 [==============================] - 1s 155ms/step - loss: 0.5173 - accuracy: 0.5905 - val_loss: 1.4975 - val_accuracy: 0.4200
# Epoch 24/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.505464480874317
#  - val_f1_(macro): 0.34691656371688445


# 8/8 [==============================] - 1s 182ms/step - loss: 0.5213 - accuracy: 0.5805 - val_loss: 1.3881 - val_accuracy: 0.4200
# Epoch 25/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.41120218579234974
#  - val_f1_(macro): 0.3639144327373307


# 8/8 [==============================] - 2s 248ms/step - loss: 0.4363 - accuracy: 0.6437 - val_loss: 1.4125 - val_accuracy: 0.3875
# Epoch 26/70
# 13/13 [==============================] - 0s 8ms/step
#  - fnc_score: 0.4849726775956284
#  - val_f1_(macro): 0.36343079823148


# 8/8 [==============================] - 2s 192ms/step - loss: 0.4357 - accuracy: 0.6086 - val_loss: 1.5963 - val_accuracy: 0.4300
# Epoch 27/70
# 13/13 [==============================] - 0s 7ms/step
#  - fnc_score: 0.5068306010928961
#  - val_f1_(macro): 0.3565710182392647


# 8/8 [==============================] - 1s 163ms/step - loss: 0.4130 - accuracy: 0.6600 - val_loss: 1.5070 - val_accuracy: 0.4325
# CPU times: user 53.9 s, sys: 1.7 s, total: 55.6 s
# Wall time: 1min 11s


# -------------------------------------------------------------
# |           |   Agree   | Disagree  |  Discuss  | Unrelated |
# -------------------------------------------------------------
# |   Agree   |     6     |     4     |    14     |     3     |
# -------------------------------------------------------------
# | Disagree  |     3     |    27     |    10     |     2     |
# -------------------------------------------------------------
# |  Discuss  |    13     |    27     |    119    |    36     |
# -------------------------------------------------------------
# | Unrelated |     6     |    25     |    84     |    21     |
# -------------------------------------------------------------



#               precision    recall  f1-score   support

#        Agree       0.21      0.22      0.22        27
#     Disagree       0.33      0.64      0.43        42
#      Discuss       0.52      0.61      0.56       195
#    Unrelated       0.34      0.15      0.21       136

#     accuracy                           0.43       400
#    macro avg       0.35      0.41      0.36       400
# weighted avg       0.42      0.43      0.41       400

